In [34]:
# spray_simple.ipynb
# simple spray deposition control program
# written by Veit Wagner
# 07.07.2023 v0.01 initial version
# 14.08.2023 v0.02 working version with cam and webcam
# =========================================================
#      load libraries etc
# =========================================================
import numpy as np
from datetime import datetime
import time
import os
from instr.asrl_pyserial import asrl_serial_class
import minimalmodbus                      # for Präzitherm TR28-3T-RS hotplate (instr/cal3300.py),   https://minimalmodbus.readthedocs.io/en/stable/
from instr.dug2005 import dug2005_class   # Sonozap Digital Ultrasonic Generator (instr/dug2005.py)
from instr.al1010 import al1010_class     # Aladdin pump controller AL-1010 (instr/al1010.py)
from instr.cal3300 import cal3300_class   # Präzitherm TR28-3T-RS hotplate (instr/cal3300.py)
from instr.pd4n5918 import pd4n5918_class # stepper motor PD4-N5918 M4204 from NanoTec (instr/pd4n5918.py)
import pyvisa
from instr.k3390 import k3390_class       # function generator for pulsed LED illumination
# --- wide jupyter window display ---
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# --------------------------------------------
def init_FG(k3390, lvl=0, f_Hz=4000.0, l_s=0.6e-6, verbose=True): # lvl=0 1st init , later lvl=1 is sufficient
    if lvl==0:
        k3390.setSYNC(False)
        k3390.vi.write("FORM:BORD SWAP")            # required by scilab_k3390_sendWF0.sci; ensure was sent before !!!  // FORMat:BORDer {NORMal|SWAPped} 
        k3390.vi.write("DATA:DAC VOLATILE,0,255,0") # be sure its exists (perhaps not needed for KE)
        k3390.setTerm50Ohm(False)
        k3390.setV(0.0, 0.02)                       # 10mVpp to 10Vpp in 50Ω; 20mVpp to 20Vpp in Hi-Z
    k3390.setFunction('USER')
    k3390.setFunction_User('VOLATILE')
    dt, n_WF0, f0_Hz, Tmax =  k3390.dt_native(par='')   # par='std' or ''      # [dt, n_WF0, f0, Tmax] = k3390_dt_native('');
    if f_Hz < 1/Tmax:
        nfac = int(np.ceil( 1/(f_Hz*Tmax)))
        dt = nfac * dt
        if verbose:
            print(f" info: enlargement of smallest dt neccesssary -> nfac = {nfac:d}")
    T = 1 / f_Hz  # [s]
    N = int(round(T/dt)) # no of points in waveform
    if verbose:
        print(f"{f_Hz:.2f} Hz -> {N:d} points in waveform")
    t = (T/N) * (np.arange(N)+0.5)  # [s] time vector of waveform
    a = np.zeros_like(t)  # (-) amplitude vector of waveform, values [-1..+1] 
    #
    # turn output off and sleep for 1 s to avoid burning LED
    outpstat = k3390.getoutput()
    k3390.output(False)
    time.sleep(0.010)      # 0.010 s
    
    # define used waveform: 
    d = 20e-6  # [s] Delay

    time.sleep(0.300)   # 0.3 s
    #l_s = 2e-6 * 0.3 # VW
    a = add_pulse(t, a, d+10e-6, l_s)

    if not np.any(a == -1.0):
        a[-1] = -1.0 # ensure -1 exists in waveform
    if not np.any(a == 1.0):
        a[-2] = 1.0 # ensure +1 exists in waveform
        
    a[:100] = -0.1    # t=0 position marker

    dac = np.int16(np.round(8191*a))
    k3390.sendWF0(dac)  # (slower) ASCII transfer
    # err = k3390_sendWF0_bin(viFG, dac); // <- needs to have scilab_k3390_sendWF0.sci loaded and linked
    #if(err ~= 0) then disp(msprintf("error %d sending waveform.", err)); end

    k3390.setFreq(f_Hz)  #*size(a,'*')/size(dac,'*'));
    #if(err ~= 0) then disp(msprintf("error %d setting frequency to %.2f Hz.", err, f)); end
    #x tFGdac = dt * size(a,'*');  // total time of waveform
    k3390.setV(0.0, 20.0)  # (V0, 2*abs(dV)+abs(VR)); // Voffs, Vampl

    time.sleep(max(2*T, 0.1))     # [s]

    k3390.output(False) # already done above
    k3390.setSYNC(False)

# --------------------------------------------
# input:   t       [s] waveform time vector
#          a       [-] waveform amplitude vector
#          t_start [s] start of pulse
#          width   [s] width of pulse
#          lvl     [-] amplitude lvl of pulse [-1..+1] (dflt=1)
# output:  a       [-] modified waveform amplitude vector
def add_pulse(t, a, t_start, width, lvl=1.0):
    a[np.logical_and(t_start <= t, t <= t_start+width)] = lvl
    return a

# --------------------------------------------
def turn_LED_on(k3390):
    k3390.output(True)
# --------------------------------------------
def turn_LED_off(k3390):
    k3390.output(False)
    
# --------------------------------------------
def clear_standby_ram():
    #os.system('C:\\Users\\jkoehling\\PycharmProjects\\data_acquisition\\EmptyStandbyList.exe') # <- must be run as admin
    os.system('C:\\Users\\spray\\Desktop\\EmptyStandbyList.exe (w Admin rights).lnk') # <- must be run as admin

# --------------------------------------------
# --- start cameras ---
import subprocess
import threading

def start_top(exp, temp, par, fn_loop_ext='', pre_outtxt=""):
    exec_str = f'start "1" cmd /C "C:\\Sapera_Cam\\GrabConsole.exe -q -a 5000 -n {par["cam_N_frames"]:d} -e {int(par["cam_f_Hz"]):d} -s {par["cam_aqs_server_name"]:s} -c {par["cam_config_file"]:s} -o {par["cam_base_output_avi-file"]+fn_loop_ext:s}"'
    print(pre_outtxt + "exec: ", exec_str)
    recording = subprocess.check_output( exec_str, shell=True)

#x def start_side(exp, temp):
#x     recording = subprocess.check_output(
#x         f'start "2" cmd /C "C:\\Sapera_Cam\\GrabConsole.exe -q -a 5000 -n 50000 -e 2000 -s Nano-M640_2 -c capture_side_4khz.ccf -o C:\\buffer\\{exp}_{temp}_side"',
#x         shell=True)
# --------------------------------------------

# --------------------------------------------
def print_log(f_log, txt):
    print(txt)
    f_log.write(txt+'\n')


In [22]:
# --- set is windows flag ---
is_win = os.name == 'nt'
# --- show available ports ---
print("--- list of found serial ports ---")
import serial.tools.list_ports as list_ports
ports = list_ports.comports()
for port, desc, hwid in sorted(ports):
    print(f"{port}: {desc} [{hwid}]")
if True:
    print("--- list of found VISA resources ---")
    rsc = pyvisa.ResourceManager()
    ls = rsc.list_resources()
    print(f"{ls}")
    rsc.close()
    del rsc

--- list of found serial ports ---
COM1: Kommunikationsanschluss (COM1) [ACPI\PNP0501\0]
COM5: Silicon Labs CP210x USB to UART Bridge (COM5) [USB VID:PID=10C4:EA60 SER=0001 LOCATION=1-4.2]
COM6: USB Serial Port (COM6) [USB VID:PID=0403:6001 SER=FTSXNG08A]
COM7: USB Serial Port (COM7) [USB VID:PID=0403:6001 SER=FTG4T3HRA]
COM8: Silicon Labs CP210x USB to UART Bridge (COM8) [USB VID:PID=10C4:EA60 SER= LOCATION=1-4.1.2]
--- list of found VISA resources ---
('USB0::0x05E6::0x3390::1415620::INSTR', 'ASRL1::INSTR', 'ASRL5::INSTR', 'ASRL6::INSTR', 'ASRL7::INSTR', 'ASRL8::INSTR')


In [23]:
# =========================================================
#      initialization of instruments
# =========================================================
use_du2 = True                # Sonozap Digital Ultrasonic Generator (instr/dug2005.py)
use_al1 = True                # Aladdin pump controller AL-1010 (instr/al1010.py)
use_c33 = True                # Präzitherm TR28-3T-RS hotplate (instr/cal3300.py)
use_pd4 = True                # stepper motor PD4-N5918 M4204 from NanoTec (instr/pd4n5918.py)
use_cam = True
use_webcam = False

if use_du2:
    asrl_no_du2 = 7 # (COM7) [USB VID:PID=0403:6001 SER=FTG4T3HRA]
    asrl = asrl_serial_class(asrl_no=asrl_no_du2, baud=38400, data_bits=8, parity=0, stop_bits=10, flow_ctrl=0, termchar_en=0)
    asrl.set_timeout(0.1) # sec
    du2 = dug2005_class(asrl)
    print(f'--- Sonozap Digital Ultrasonic Generator (COMport {asrl_no_du2}) ---')
    print('Ping:', du2.ping())
    print('Software ver:', du2.get_softw_ver())
    print('System state:', du2.get_system_state())

if use_al1:
    asrl_no_al1 = 6  # COM6: #com='/dev/ttyUSB2'  #com='COM6' # (COM6) [USB VID:PID=0403:6001 SER=FTSXNG08A]
    asrl = asrl_serial_class(asrl_no=asrl_no_al1, baud=19200, data_bits=8, parity=0, stop_bits=10, flow_ctrl=0, termchar_en=1, termchar=13)
    asrl.set_timeout(0.05)
    al1 = al1010_class(asrl, adr=1)
    print(f'--- Aladdin pump controller AL-1010 (COMport {asrl_no_al1}) ---')
    print( f"Version of pump {al1.get_dflt_adr()} : {al1.firmware_version()}" )
    
if use_c33:
    asrl_no_c33 = 8  # COM6: #com='/dev/ttyUSB2'  #com='COM6' # (COM8) [USB VID:PID=10C4:EA60 SER= LOCATION=1-4.1.2]
    com     = f"COM{asrl_no_c33:d}" # Windows
    adr     = 1  #slave address
    mdbs = minimalmodbus.Instrument(com, adr, mode='rtu') # port name, slave address (in decimal)
    mdbs.serial.baudrate = 19200 # baud, for more parameters see https://minimalmodbus.readthedocs.io/en/stable/usage.html
    mdbs.serial.timeout  = 0.5   # [s] timeout (dflt = 0.05 s)
    c33 = cal3300_class(mdbs, adr=adr)
    typ, txt = c33.get_model()
    print(f'--- Präzitherm TR28-3T-RS hotplate (instr/cal3300.py) (COMport {asrl_no_c33}) ---') 
    print( f"Version of T ctrl : '{txt}' (0x{typ:02X})" )

if use_pd4:
    asrl_no_pd4 = 5  # COM5: #com='/dev/ttyUSB2'  #com='COM5' # (COM5) [USB VID:PID=10C4:EA60 SER=0001 LOCATION=1-4.1.2]
    devser   = f"COM{asrl_no_pd4:d}" if is_win else '/dev/ttyUSB0'
    pd4_mids = [1] #[2] # list of motors
    asrl     = asrl_serial_class(devser=devser, baud=115200, data_bits=8, parity=0, stop_bits=10, flow_ctrl=2, termchar_en=1, termchar=13)
    pd4      = pd4n5918_class(asrl, mids=pd4_mids)
    print(f'--- stepper motor PD4-N5918 M4204 from NanoTec (instr/pd4n5918.py) (COMport {asrl_no_pd4}) ---')
    mid = pd4_mids[0] # use 1st motor in list, which has 0.001 mm per step
    pd4.set_mid_and_mm_per_step(mid=mid, mm_per_step=0.001)
    print(f"motor {mid} status : {pd4.Status_auslesen(mid):X} ,  position : {pd4.get_current_pos_abs_mm()} mm")

if use_cam:
    rsc = pyvisa.ResourceManager()
    instr_txt = 'USB0::0x05E6::0x3390::1415620::INSTR' # Keithley 3390 Function Generator
    vi = rsc.open_resource(instr_txt) #) #'GPIB0::12::INSTR')
    k3390 = k3390_class(vi)
    print(f'--- function generator ---')
    #k3390.output(True)
    #V0, Vp = k3390.getV()
    #print(f" Voltages: V0 = {V0} V, Vp = {Vp} V")
    init_FG(k3390, f_Hz=60.0, l_s=1.0e-6)


--- Sonozap Digital Ultrasonic Generator (COMport 7) ---
Ping: (b'', 0)
Software ver: 5.01
System state: 1
--- Aladdin pump controller AL-1010 (COMport 6) ---
Version of pump 1 : NE1010V3.922
--- Präzitherm TR28-3T-RS hotplate (instr/cal3300.py) (COMport 8) ---
Version of T ctrl : '33/9300 SSD / SSD' (0x102)
--- stepper motor PD4-N5918 M4204 from NanoTec (instr/pd4n5918.py) (COMport 5) ---
motor 1 status : A1 ,  position : 115.0 mm
--- function generator ---
 info: enlargement of smallest dt neccesssary -> nfac = 11
60.00 Hz -> 189394 points in waveform


In [35]:
# =========================================================
#      deposition run with given parameters
# =========================================================
par = {
    'user':                    'VW,TS',
    'sample':                  'Perowskite Precursor on cleaned Si',
    'comment':                 'new substrate',
    'start date':               datetime.now().strftime("%d.%m.%Y %H:%M:%S %z"),
    'sonic_power_perc':              57,
    'syring_diameter_mm':           20.0, #2.2,
    'syring_pump_rate_mL_min':      0.001, #0.1
    'syring_pump_start_delay_s':    0.1,  # [sec] delay of pump start after motor start
    'syring_pump_depos_time_s':    20,  # [sec] pump active time pumping
    'temperature_sample_C':        60.0,
    'nozzle_run_no_of_repeats':       1,  # 1=single run only
    'nozzle_run_period_s':         10.0,  # [sec] time from nozzle start to previous repeat nozzle start (only needed if not single run)
    'nozzle_run_both_directions':  False,  # flag if deposition on both directions (only needed if not single run)
    'nozzle_pos_start_mm':        115.0,
    'nozzle_pos_end_mm':          0.0,
    'nozzle_pos_speed_depos_mm_s':  5.0,
    'nozzle_pos_speed_mm_s':       15.0,
    'nozzle_pos_acc_mm_s2':         5.0,
    'cam_base_output_avi-file':   r"C:\buffer\xxPerowskite_Precursor_T60_416_384_f1k_13", # w/o ext , (dflt=c:\buffer\grabbed)
    'cam_f_Hz':                    1000.0/1, #500.0, #280.0, #4000.0,
    'cam_N_frames':               (7+22)*1000//1, #8000,   # no of frames to grab
    'cam_LED_pulse_len_s':       1.1e-6, #0.6e-6,
    'cam_start_delay_s':            0.5,  # [sec] delay of camera start after motor start
    'cam_config_file':     'T_Nano-M1280_TriggerMode_On_416x384.ccf', #'T_Nano-M1280_TriggerMode_On_640x512.ccf', #'T_Nano-M1280_TriggerMode_On_1280x1024.ccf', #'capture_4khz.ccf', #'T_Nano-M1280_Default_Default_VW_2022.04.21.ccf',  # w/o path -> is in "%SAPERADIR%\CamFiles\User\" ; SAPERADIR = C:\Program Files\Teledyne DALSA\Sapera
    'cam_aqs_server_name': 'Nano-M1280_1',   #'Nano-M1280_2',   # acquisition server name
}

In [36]:
dt00, ts00 = datetime.now(), time.time()
#print(f"start timestamps: ts00 = {ts00}, dt00.timestamp() = {dt00.timestamp()}, ts00-dt00.timestamp() = {ts00-dt00.timestamp()}")
assert abs(ts00 - dt00.timestamp()) < 0.01, f"too large timestamp difference!  start timestamps: ts00 = {ts00}, dt00.timestamp() = {dt00.timestamp()}, ts00-dt00.timestamp() = {ts00-dt00.timestamp()}"
t00 = dt00.timestamp() # <- same as from time.time()
par['start date'] = dt00.strftime("%d.%m.%Y %H:%M:%S.%f %z") # document actual start time here
par['start date timestamp'] = t00
f_log = open(par['cam_base_output_avi-file']+"_memo.txt", "w")
print_log(f_log, f"{par}")
print_log(f_log, "")

# --- set temperature ---
if use_c33:
    if False:
        print_log( f"{time.time()-t00:7.3f}s  setting temperature setpoint to : {par['temperature_sample_C']} C  ;  current temperature : {c33.readT_C()} C" )
        c33.set_setpoint(par['temperature_sample_C']) # set target T in C
        for i in range(20): #range(100000):
            if abs(c33.readT_C() - par['temperature_sample_C']) < 0.5:
                break
            time.sleep(0.5)
    print_log(f_log, f"{time.time()-t00:7.3f}s  temperature set:  setpoint      : {c33.read_setpoint()} C  ;  current temperature : {c33.readT_C()} C" )

# --- start ultrasonic nozzle ---
if use_du2:
    du2.set_system_state(du2.running)
    print_log(f_log, f"{time.time()-t00:7.3f}s  Ultrasonic nozzle System state: {du2.get_system_state()}")
    time.sleep(1)
    du2.set_power_level( min(100, max(0, int(round(par['sonic_power_perc'])) ) ) )
    time.sleep(0.3)
    print_log(f_log, f"{time.time()-t00:7.3f}s  Ultrasonic nozzle Power level :  target: {par['sonic_power_perc']} % , current value: {du2.get_power_level()} % {du2.get_power()} W  ; Frequency: {du2.get_frequency()} Hz")

# --- set syring parameter ---
if use_al1:
    al1.stop_pump() # stop pumping
    al1.set_diameter(par['syring_diameter_mm']) # [mm] diameter
    al1.set_pumping_rate(par['syring_pump_rate_mL_min'], 'MM') # allowed units: 'UM'=uL/min, 'MM'=mL/min, 'UH'=uL/hr, 'MH'=mL/hr
    al1.set_volume(0) # 0=cont. pumping
    al1.set_pumping_direction(al1.infuse) # =al1.infuse, al1.withdraw, al1.reverse

# --- move nozzle to start position ---
if use_pd4:
    print_log(f_log, f"{time.time()-t00:7.3f}s  Moving to start position: {par['nozzle_pos_start_mm']} mm")
    pd4.move_to_pos_abs_mm(par['nozzle_pos_start_mm'], par['nozzle_pos_speed_mm_s'], par['nozzle_pos_acc_mm_s2'])
    while not pd4.ready(): # wait finish movement
            time.sleep(0.02)

# --- program LED pulse width ---
if use_cam:
    k3390.output(False)
    k3390.setSYNC(False)
    t00 = time.time()
    #print_log(f_log, f"start {k3390.vi.get_visa_attribute(pyvisa.constants.VI_ATTR_TMO_VALUE)}")
    init_FG(k3390, lvl=1, f_Hz=par['cam_f_Hz'], l_s=par['cam_LED_pulse_len_s'], verbose=True)
    #print_log(f_log, f"back  {time.time()-t00} {k3390.vi.get_visa_attribute(pyvisa.constants.VI_ATTR_TMO_VALUE)}")
    time.sleep(1.5)
    dummy = k3390.getoutput() #print_log(f_log, f"{k3390.getoutput()}" ) # <- needed to avoid TMO error ??
    k3390.output(True) # LED pulses on
    k3390.setSYNC(True) # Camera Trigger on
    #print_log(f_log, "finished.")
    

cam_end_before_syring = (par['cam_start_delay_s'] + par['cam_N_frames'] / par['cam_f_Hz'] ) < (par['syring_pump_start_delay_s'] + par['syring_pump_depos_time_s'])

# === repetition loop ===
for i_rep in range(par['nozzle_run_no_of_repeats']):
    print_log(f_log, f"{time.time()-t00:7.3f}s  --- Repetition i_rep = {i_rep} ---")

    # --- move nozzle to start position ---
    if use_pd4 and ((not par['nozzle_run_both_directions']) and (i_rep > 0)):
        pd4.move_to_pos_abs_mm(par['nozzle_pos_start_mm'], par['nozzle_pos_speed_mm_s'], par['nozzle_pos_acc_mm_s2'])
        while not pd4.ready(): # wait finish movement
                time.sleep(0.02)

    # --- wait for deposition start ---
    if (i_rep == 0) or (time.time() > t_n):
        t_n = time.time()    # remember time of initial start or if given period too short 
    else:
        while time.time() < t_n: # wait finish movement
                time.sleep(0.001)
        
    print_log(f_log, f"{time.time()-t00:7.3f}s  use t_n-t00 = {t_n-t00} s")

    # --- move nozzle for deposition ---
    if use_pd4:
        pos = par['nozzle_pos_end_mm'] if (not par['nozzle_run_both_directions'] ) or (i_rep % 2 == 0)  else  par['nozzle_pos_start_mm']
        pd4.move_to_pos_abs_mm(pos, par['nozzle_pos_speed_depos_mm_s'], par['nozzle_pos_acc_mm_s2'])
        print_log(f_log, f"{time.time()-t00:7.3f}s  Start deposition movement from {pd4.get_current_pos_abs_mm()} mm towards {pos} mm")

    # --- wait loop syring and camera start ---
    al1_started, cam_started = False, False
    t_x_al1, t_x_cam         = t_n + par['syring_pump_start_delay_s'], t_n + par['cam_start_delay_s']
    while (use_al1 and not al1_started) or (use_cam and not cam_started):
        t = time.time()
        if (use_al1 and not al1_started) and t_x_al1 <= t:
            # --- start syring pump ---
            al1.start_pump()
            t_tmp = time.time()-t00
            s = f" , current motorposition {pd4.get_current_pos_abs_mm()} mm" if use_pd4  else ""
            print_log(f_log, f"{t_tmp:7.3f}s  Start pump{s}")
            al1_started = True
        if (use_cam and not cam_started) and t_x_cam <= t:
            # --- start camera(s) ---
            k3390.output(True) # LED pulses on
            #k3390.setSYNC(True) # Camera Trigger on
            exp, t_now = None, None
            fn_loop_ext = '' if par['nozzle_run_no_of_repeats']==1 else f'_{i_rep:02d}'
            t_tmp = time.time()-t00
            thr_cam = threading.Thread(target=start_top, args=(exp, t_now, par, fn_loop_ext, f"{t_tmp:7.3f}s  " ))
            #x s = threading.Thread(target=start_side, args=(exp, t_now))
            # sleep in between, otherwise camera servers are not properly detected
            thr_cam.start()
            #time.sleep(1.5)
            #x s.start()
            ##time.sleep(1)
            k3390.setSYNC(True) # Camera Trigger on
            t_tmp = time.time()-t00
            s = f" , current motorposition {pd4.get_current_pos_abs_mm()} mm" if use_pd4  else ""
            print_log(f_log, f"{t_tmp:7.3f}s  Start camera{s}")
            cam_started = True

    # --- wait for syring pump and recording completed ---
    if use_cam and cam_end_before_syring:
        # wait for recording to be complete
        thr_cam.join()
        #x s.join()
        t_tmp = time.time()-t00
        s = f" , current motorposition {pd4.get_current_pos_abs_mm()} mm" if use_pd4  else ""
        print_log(f_log, f"{t_tmp:7.3f}s  Stop camera{s}")
        k3390.setSYNC(False) # Camera Trigger off
        turn_LED_off(k3390)  # turn LED off
        cam_started = False

    if use_al1:
        # --- wait for syring stop ---
        t_x_al1 = t_x_al1 + par['syring_pump_depos_time_s']
        while time.time() < t_x_al1: # wait finish mov
            time.sleep(0.001)
        al1.stop_pump()
        t_tmp = time.time()-t00
        s = f" , current motorposition {pd4.get_current_pos_abs_mm()} mm" if use_pd4  else ""
        print_log(f_log, f"{t_tmp:7.3f}s  Stop pump{s}")

    if use_cam and cam_started:
        # wait for recording to be complete (if not already done)
        thr_cam.join()
        #x s.join()
        t_tmp = time.time()-t00
        s = f" , current motorposition {pd4.get_current_pos_abs_mm()} mm" if use_pd4  else ""
        print_log(f_log, f"{t_tmp:7.3f}s  Stop camera{s}")
        k3390.setSYNC(False) # Camera Trigger off
        turn_LED_off(k3390)  # turn LED off
        cam_started = False

    # --- wait motor movement finished ---
    if use_pd4:
        while not pd4.ready(): # wait finish movement
            time.sleep(0.02)

    if use_c33:
        print_log(f_log, f"{time.time()-t00:7.3f}s  temperature set:  setpoint      : {c33.read_setpoint()} C  ;  current temperature : {c33.readT_C()} C" )

    # --- set start time for next repetition ---
    t_n += par['nozzle_run_period_s'] 
        
# === end of repetition loop ===

# --- move nozzle back to start position ---
if use_pd4:
    pd4.move_to_pos_abs_mm(par['nozzle_pos_start_mm'], par['nozzle_pos_speed_mm_s'], par['nozzle_pos_acc_mm_s2'])
    while not pd4.ready(): # wait finish movement
            time.sleep(0.02)

if use_al1:
    al1.stop_pump() # stop pumping , for safety

if use_du2:
    du2.set_system_state(du2.stopped)

if use_c33:
    print_log(f_log, f"{time.time()-t00:7.3f}s  temperature set:  setpoint      : {c33.read_setpoint()} C  ;  current temperature : {c33.readT_C()} C" )

print_log(f_log, f"{time.time()-t00:7.3f}s  finished.")
f_log.close() # close log file .._memo.txt

{'user': 'VW,TS', 'sample': 'Perowskite Precursor on cleaned Si', 'comment': 'new substrate', 'start date': '14.08.2023 19:38:40.907648 ', 'sonic_power_perc': 57, 'syring_diameter_mm': 20.0, 'syring_pump_rate_mL_min': 0.001, 'syring_pump_start_delay_s': 0.1, 'syring_pump_depos_time_s': 20, 'temperature_sample_C': 60.0, 'nozzle_run_no_of_repeats': 1, 'nozzle_run_period_s': 10.0, 'nozzle_run_both_directions': False, 'nozzle_pos_start_mm': 115.0, 'nozzle_pos_end_mm': 0.0, 'nozzle_pos_speed_depos_mm_s': 5.0, 'nozzle_pos_speed_mm_s': 15.0, 'nozzle_pos_acc_mm_s2': 5.0, 'cam_base_output_avi-file': 'C:\\buffer\\xxPerowskite_Precursor_T60_416_384_f1k_13', 'cam_f_Hz': 1000.0, 'cam_N_frames': 29000, 'cam_LED_pulse_len_s': 1.1e-06, 'cam_start_delay_s': 0.5, 'cam_config_file': 'T_Nano-M1280_TriggerMode_On_416x384.ccf', 'cam_aqs_server_name': 'Nano-M1280_1', 'start date timestamp': 1692034720.907648}

  0.001s  temperature set:  setpoint      : 60.0 C  ;  current temperature : 60.1 C
  0.085s  Ultra

In [38]:
# =========================================================
#      close instruments
# =========================================================
if use_du2:
    du2.asrl.close()

if use_al1:
    al1.asrl.close()
    
if use_c33:
    c33.mdbs.serial.close()

if use_pd4:
    pd4.asrl.close()

if use_cam:
    k3390.vi.close()
    rsc.close()

In [ ]:
# =========================================================
#      interactive operation of Sonozap Digital Ultrasonic Generator (instr/dug2005.py)
# =========================================================
print('set System state running.')
du2.set_system_state(du2.running)
print('System state:', du2.get_system_state())

time.sleep(1)
print('Frequency (Hz) :', du2.get_frequency())
print('Power     (W)  :', du2.get_power())
plvl = du2.get_power_level()
print('Power level (%):', plvl)
time.sleep(1)
print('set power level')
du2.set_power_level( 80) #int(min(100, max(0, plvl-5))) )
time.sleep(0.3)
print('Power level (%):', du2.get_power_level())
time.sleep(1)

print('set System state stopped.')
du2.set_system_state(du2.stopped)
print('System state:', du2.get_system_state())
time.sleep(1)
print('Power     (W)  :', du2.get_power())

print('Request Fault  :', du2.request_fault())

In [39]:
du2.set_system_state(du2.running)

0

In [ ]:
# =========================================================
#      interactive operation of Aladdin pump controller AL-1010 (instr/al1010.py)
# =========================================================
# --- set/get inner diameter ---
D_mm = al1.get_diameter()
al1.set_diameter(D_mm)
print( f"Syringe inner diameter set to {D_mm} mm")

# --- set/get pumping rate ---
rate, unit = al1.get_pumping_rate()
al1.set_pumping_rate(rate, unit)
print( f"Pumping rate set to {rate} {unit} ('UM'=uL/min, 'MM'=mL/min, 'UH'=uL/hr, 'MH'=mL/hr)")

# --- setting volume to be dispensed or continuous pumping ---
vol, unit = al1.get_volume()
al1.set_volume(vol) # 0=cont. pumping
print( f"Volume set to {vol} {unit} ('UL' microliters, 'ML' milliliters )")

# --- setting pumping direction ---
pump_dir = al1.get_pumping_direction()   # 'INF', 'WDR' for infuse, withdraw
al1.set_pumping_direction(pump_dir) # =al1.infuse, al1.withdraw, al1.reverse
#al1.set_pumping_direction(al1.infuse)
print( f"set pumping direction to {pump_dir} ('INF' infuse, 'WDR' withdraw )")

In [40]:
# --- start/stop pumping ---
print("Start pumping ..")
al1.start_pump()
time.sleep(1)
#time.sleep(180)

Start pumping ..


In [41]:
print("Stop pumping ..")
al1.stop_pump()

Stop pumping ..


In [ ]:
# --- setting pumping direction ---
pump_dir = al1.withdraw   # 'INF', 'WDR' for infuse, withdraw
al1.set_pumping_direction(pump_dir) # =al1.infuse, al1.withdraw, al1.reverse
#al1.set_pumping_direction(al1.infuse)
print( f"set pumping direction to {pump_dir} ('INF' infuse, 'WDR' withdraw )")

al1.set_pumping_rate(0.7, "MM")

# --- start/stop pumping ---
print("Start pumping ..")
al1.start_pump()
time.sleep(10)
#time.sleep(180)
print("Stop pumping ..")
al1.stop_pump()

In [38]:
# =========================================================
#      interactive operation of Präzitherm TR28-3T-RS hotplate (instr/cal3300.py)
# =========================================================
N = 1
if True: # --- set target temperature ---
    target_T_C = 60.0
    T_old = c33.read_setpoint()
    c33.set_setpoint(target_T_C)
    print( f"old setpoint : {T_old} C  -> new setpoint : {target_T_C} C" )
    N = 20
    
t00 =time.time()
for i in range(N):
    S = c33.read_setpoint()
    T = c33.readT_C()
    print( f"{time.time() - t00:6.2f} s  setpoint : {S} C  ; temperature : {T} C" )
    time.sleep(0.5)

old setpoint : 55.0 C  -> new setpoint : 60.0 C
  0.09 s  setpoint : 60.0 C  ; temperature : 55.0 C
  0.66 s  setpoint : 60.0 C  ; temperature : 55.0 C
  1.23 s  setpoint : 60.0 C  ; temperature : 55.0 C
  1.79 s  setpoint : 60.0 C  ; temperature : 55.0 C
  2.36 s  setpoint : 60.0 C  ; temperature : 55.0 C
  2.96 s  setpoint : 60.0 C  ; temperature : 55.0 C
  3.55 s  setpoint : 60.0 C  ; temperature : 55.0 C
  4.12 s  setpoint : 60.0 C  ; temperature : 55.0 C
  4.72 s  setpoint : 60.0 C  ; temperature : 55.0 C
  5.27 s  setpoint : 60.0 C  ; temperature : 55.0 C
  5.85 s  setpoint : 60.0 C  ; temperature : 55.0 C
  6.43 s  setpoint : 60.0 C  ; temperature : 55.0 C
  6.98 s  setpoint : 60.0 C  ; temperature : 55.0 C
  7.54 s  setpoint : 60.0 C  ; temperature : 55.0 C
  8.12 s  setpoint : 60.0 C  ; temperature : 55.0 C
  8.67 s  setpoint : 60.0 C  ; temperature : 55.0 C
  9.27 s  setpoint : 60.0 C  ; temperature : 55.0 C
  9.83 s  setpoint : 60.0 C  ; temperature : 55.0 C
 10.41 s  setpoi

In [37]:
# =========================================================
#      interactive operation of stepper motor PD4-N5918 M4204 from NanoTec (instr/pd4n5918.py)
# =========================================================
if True: # --- move to position ---
    end_mm, speed_mm_s, acc_mm_s2 = 0, 5, 5.0 # (-2) 0..346 (348) mm
    pos_mm = pd4.get_current_pos_abs_mm()
    print(f"current pos {pos_mm} mm ; target pos {end_mm} mm ; speed {speed_mm_s} mm/s : acceleration {acc_mm_s2} mm/s2 ..  (stop motor with: pd4.stop() ; set position with: pd4.set_is_at_pos_abs_mm(pos)")
    t00 = time.time()
    pd4.move_to_pos_abs_mm(end_mm, speed_mm_s, acc_mm_s2)
    while not pd4.ready():
        time.sleep(0.02)
    print(f"Dt = {time.time() - t00:.3f} s")
if False: # --- set position ---
    pd4.set_is_at_pos_abs_mm(115.0) # [mm]
# -- get status and position ---
mot_stat = pd4.Status_auslesen(pd4.mid)
pos_mm = pd4.get_current_pos_abs_mm()
print(f"motor {pd4.mid}: status={mot_stat:X}, position = {pos_mm} mm")
print("finished.")

current pos 115.0 mm ; target pos 0 mm ; speed 5 mm/s : acceleration 5.0 mm/s2 ..  (stop motor with: pd4.stop() ; set position with: pd4.set_is_at_pos_abs_mm(pos)
Dt = 23.893 s
motor 1: status=A1, position = 0.0 mm
finished.


In [ ]:
pd4.stop()

In [9]:
(1e9/10) /(640*512)

305.17578125

In [62]:
# =========================================================
#      interactive operation of camera Nano M1280 and Keithley 3390 Function Generator
# =========================================================
if True:
    k3390.output(False)
    k3390.setSYNC(False)
    t00 = time.time()
    print("start", k3390.vi.get_visa_attribute(pyvisa.constants.VI_ATTR_TMO_VALUE))
    #init_FG(k3390, lvl=1, f_Hz=2500.0, l_s=0.6e-6, verbose=True)
    init_FG(k3390, lvl=1, f_Hz=1*1000.0+0*par['cam_f_Hz'], l_s=1*1.1e-6+0*par['cam_LED_pulse_len_s'], verbose=True)
    print("back", time.time()-t00, k3390.vi.get_visa_attribute(pyvisa.constants.VI_ATTR_TMO_VALUE))
    time.sleep(1.5)
    print( f"ouput state = {k3390.getoutput()}" ) # <- needed to avoid TMO error ??
    k3390.output(True) # LED pulses on
    k3390.setSYNC(True) # Camera Trigger on
    print("finished.")

start 2000
1000.00 Hz -> 125000 points in waveform
back 5.4807751178741455 2000
ouput state = False
finished.


In [58]:
k3390.output(False)
k3390.setSYNC(False)

In [26]:
640*512/(400**2), 2**4 * 5**2, 5**4, 24*26, 26*16, 24*16, (1280-416)/2, (1024-384)/2

(2.048, 400, 625, 624, 416, 384, 432.0, 320.0)

In [6]:
def clear_standby_ram():
    # os.system('C:\\Users\\jkoehling\\PycharmProjects\\data_acquisition\\EmptyStandbyList.exe') # <- must be run as admin
    # ( https://thegeekpage.com/how-to-enable-standard-users-to-run-a-program-with-admin-rights-in-windows-11-or-10/ )
    #os.system('"C:\\Users\\franzmarc\\Desktop\\EmptyStandbyList.exe (w Admin rights).lnk"') # <- must be run as admin
    os.system('"C:\\Users\\spray\\Desktop\\EmptyStandbyList.exe (w Admin rights).lnk"') # <- must be run as admin

# clear RAM
clear_standby_ram()

In [30]:
# --- start cameras ---
k3390.output(True) # LED pulses on
k3390.setSYNC(True) # Camera Trigger on

exp, t_now = None, None
par['cam_base_output_avi-file'] =  r"C:\buffer\testvw_416_384_f1k_01_dark"
t = threading.Thread(target=start_top, args=(exp, t_now, par, ''))
#x s = threading.Thread(target=start_side, args=(exp, t_now))

# sleep in between, otherwise camera servers are not properly detected
t.start()
time.sleep(1.5)
#x s.start()
# start LED
#turn_LED_on(k3390)  # turn LED on
time.sleep(1)
k3390.setSYNC(True) # Camera Trigger on

# wait for recording to be complete
t.join()
#x s.join()

time.sleep(1)
k3390.setSYNC(False) # Camera Trigger off
k3390.output(False) # turn_LED_off(k3390)  # turn LED off

exec:  start "1" cmd /C "C:\Sapera_Cam\GrabConsole.exe -q -a 5000 -n 14500 -e 1000 -s Nano-M1280_1 -c T_Nano-M1280_TriggerMode_On_416x384.ccf -o C:\buffer\testvw_416_384_f1k_01_dark"


In [57]:
k3390.output(True) # LED pulses 
k3390.setSYNC(True) # Camera Trigger on


# Program description
## Concept
### Instruments
 - Sonozap Digital Ultrasonic Generator    (instr/dug2005.py)
 - Aladdin pump controller AL-1010     (instr/al1010.py)
 - Präzitherm TR28-3T-RS hotplate     (instr/cal3300.py)
 - stepper motor PD4-N5918 M4204 from NanoTec     (instr/pd4n5918.py)
 - Keithley 3390 Function Generator     (instr/k3390.py)
 - Nano M1280 camera (<- Sapera CamExpert program)
 - PeakTech DC Power Supply 6080
 - self-mode LED drive electronics (w/ BUZ10 FET)

###  Program functionality
 - initialization of instruments
 - interactive operation of individual instruments (for testing)
 - deposition run with given parameters
 
 individual functionality is accessable in individuall jupyter notebook cells
 
### Technical memos
- check git status (in ~/py/meas/ ):
~~~
    git status
~~~

- store to git server (in ~/py/meas/ ):
~~~
    git add ...
    git commit -m "..cmt.."
    git push origin main
~~~
- 
get latest from git server (in ~/py/meas/ ):
~~~
    git pull
~~~
